# Free Energy Calculation applied to HCP dataset
*Cyril Rommens, s12495719, masterproject MSc Physics and Astronomy: Computational Physics of Complex Systems*

**Introduction**
In this notebook, we will use the free energy calculation as explored in the 'Energy_calculation.ipynb'-file following Knill's theorisation on the free energy of a simplicial complex. We will apply this calculation to data from the Human Connectome Project. This data is given in pairwise connectionmatrices, which represent interactions between important regions in the brain. Using the code in 'Connection_matrix.ipynb' we will convert this matrix into the complete connection matrix, including connections between all simplices in the simplicial complex, as required by Knill's theorisation.

**Import necessities**

In [4]:
# Import libraries
import networkx as nx
import numpy as np
from itertools import combinations
import matplotlib.pyplot as plt
import pandas as pd
import glob
import timeit
import os

In [16]:
# Import background functions
%run "./Background Scripts/functions.py"

In [6]:
# Importing all matrices to generate averaged data with Numpy or Pandas
matrices = [np.genfromtxt(file) for file in glob.glob('./1000_Functional_Connectomes/Connectivity matrices/*_matrix_file.txt')]

**Compute the connection probability matrix $L_p$**

In [7]:
L_p = np.mean(matrices, axis=0)

**Generate the clique_complex** from a desired subject in the dataset, given a certain threshold and maximum clique size. If the maximum clique size gets too large, the computational cost will increase significantly so for now do not go above 3. **Note** the threshold value does not have any physical meaning for now. Once everything works, finding a physical meaning for the sparseness of the graph is an interesting and important topic. 

In [17]:
connection_matrices = []
clique_complex_list = []
threshold = 0.9  # Set your threshold
max_clique_size = 3  # Set the maximum size of the cliques

for matrix in matrices:
    clique_complex = build_clique_complex(matrix, threshold, max_clique_size)
    clique_complex_list.append(clique_complex)
    clique_complex = sorted(clique_complex)
    overlap_matrix = generate_overlap_matrix(clique_complex)
    connection_matrices.append(overlap_matrix)

clique_complex_list_complete = flatten_list(clique_complex_list)

In [23]:
removed_rows_clique_complex = []

for i in range(0, len(clique_complex_list)):
    removed_sets_list = sets_difference(clique_complex_list_complete,clique_complex_list[i])
    removed_rows_clique_complex.append(removed_sets_list)

In [24]:
decompressed_matrix = decompress(matrices[1], len(clique_complex_list_complete), removed_rows_clique_complex[1])

TypeError: unsupported operand type(s) for +: 'set' and 'int'

**Compute the inverse connection matrix ${L_i}^{-1}$**

In [40]:
# Invert matrices for complete dataset
L_inverse_list = []
removed_rows_list = []

for matrix in connection_matrices:
    result_matrix, removed_rows, inverse_matrix = inverse_matrix_generator(matrix)
    L_inverse_list.append(inverse_matrix)
    removed_rows_list.append(removed_rows)

**Decompress the inverse matrices for later use, since we need to multiply them with the $L_p$ matrix and thus should be of equal dimensions.**

In [43]:
L_inverse_decompressed_list = []

for matrix in range(0, len(L_inverse_list)):
    decompressed_matrix = decompress(L_inverse_list[matrix], 14, removed_rows_list[matrix])
    L_inverse_decompressed_list.append(decompressed_matrix)

**Compute the internal energy, entropy and Helmholtz free energy from the dataset**

In [47]:
# Compute the functionals from the dataset
U_list, S_list, F_list = functionals(matrices, L_inverse_decompressed_list, L_p, 293)

ValueError: operands could not be broadcast together with shapes (6,6) (177,177) 